In [6]:
import mysql.connector as mysql
from mysql.connector import Error

In [9]:
# Database credentials (replace with your details)
hostname='wildfire-watch1.mysql.database.azure.com'
port=3306
username='tkirk'
password='Wildfire507'
ssl_ca = "/DigiCertGlobalRootCA.crt (1).pem"  # Required for Azure MySQL SSL connection

try:
    # Establish connection
    conn = mysql.connect(
        host=hostname,
        user=username,
        password=password,
        ssl_ca=ssl_ca  # Secure connection
    )

    if conn.is_connected():
        print("✅ Successfully connected to MySQL database")

except Error as e:
    print(f"❌ Error: {e}")



✅ Successfully connected to MySQL database
